<a href="https://colab.research.google.com/github/BrunaFerreira/Coursera_AI_for_Medical_Diagnosis/blob/master/Lab3_Patient_Overlap_and_Data_Leakage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Aluna : Bruna Ferreira

# Densenet
* AI for Medicine Course 1 Week 1 lecture exercises
* AI4M_C1_W1_lecture_ex_04


# Sobreposição do paciente (Overlap) e vazamento de dados (Data Leakage)

A sobreposição de pacientes em dados médicos é parte de um problema mais geral no aprendizado de máquina chamado <b><i> Data Leakage </i> - vazamento de dados </b>. Para identificar a sobreposição do paciente, você pode verificar se o ID de um paciente aparece tanto no conjunto de treinamento quanto no conjunto de teste, assim como no conjunto de validação. 

In [2]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
sns.set()

### Leitura dos dados

In [3]:
# Leitura do arquivo csv contendo dados de treinamento
train_df = pd.read_csv("/content/drive/MyDrive/0_Cursos/1. Coursera/AI Medicine /Code/data/images/nih/train-small.csv")
# Mostrando 5 primeiras linhas
print(f'Há {train_df.shape[0]} linhas e  {train_df.shape[1]} colunas no conjunto de treinamento')
train_df.head()

Há 1000 linhas e  16 colunas no conjunto de treinamento


,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00008270_015.png,0,0,0,0,0,0,0,0,0,0,0,8270,0,0,0
1,00029855_001.png,1,0,0,0,1,0,0,0,1,0,0,29855,0,0,0
2,00001297_000.png,0,0,0,0,0,0,0,0,0,0,0,1297,1,0,0
3,00012359_002.png,0,0,0,0,0,0,0,0,0,0,0,12359,0,0,0
4,00017951_001.png,0,0,0,0,0,0,0,0,1,0,0,17951,0,0,0


In [4]:
# Leitura do arquivo csv contendo dados de validação
valid_df = pd.read_csv("/content/drive/MyDrive/0_Cursos/1. Coursera/AI Medicine /Code/data/images/nih/valid-small.csv")
# Mostrando 5 primeiras linhas
print(f'Há {valid_df.shape[0]} linhas e {valid_df.shape[1]} colunas no conjunto de validação')
valid_df.head()

Há 200 linhas e 16 colunas no conjunto de validação


,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00008270_015.png,0,0,0,0,0,0,0,0,0,0,0,8270,0,0,0
1,00029855_001.png,1,0,0,0,1,0,0,0,1,0,0,29855,0,0,0
2,00001297_000.png,0,0,0,0,0,0,0,0,0,0,0,1297,1,0,0
3,00012359_002.png,0,0,0,0,0,0,0,0,0,0,0,12359,0,0,0
4,00017951_001.png,0,0,0,0,0,0,0,0,1,0,0,17951,0,0,0


### Comparando as colunas PatientId nos conjuntos de treinamento e validação

1. Extraia as IDs dos pacientes dos conjuntos de treinamento e validação
2. Converta essas matrizes de números em tipos de dados `set ()` para facilitar a comparação
3. Identifique a sobreposição do paciente na interseção dos dois conjuntos.

In [5]:
# IDs dos pacientes nos dados de treinamento
ids_train = train_df.PatientId.values
# IDs dos pacientes nos dados de validação
ids_valid = valid_df.PatientId.values

In [6]:
ids_train_set = set(ids_train)
print(f'Há {len(ids_train_set)} IDs de pacientes únicos no conjunto de treinamento.')

ids_valid_set = set(ids_valid)
print(f'Há {len(ids_valid_set)} IDs de pacientes únicos no conjunto de validação')

Há 928 IDs de pacientes únicos no conjunto de treinamento.
Há 199 IDs de pacientes únicos no conjunto de validação


In [7]:
# identificando sobreposição de pacientes procurando interseção nos conjuntos
patient_overlap = list(ids_train_set.intersection(ids_valid_set))
n_overlap = len(patient_overlap)
print(f'Há {n_overlap} IDs de pacientes nos conjuntos de treinamento e validação')
print('')
print(f'Pacientes que estão no conjunto de treinamento e validação:')
print(f'{patient_overlap}')

Há 197 IDs de pacientes nos conjuntos de treinamento e validação

Pacientes que estão no conjunto de treinamento e validação:
[17920, 15360, 17417, 28682, 16397, 27665, 25628, 22559, 22047, 17951, 17443, 8741, 8230, 22572, 25645, 28208, 11315, 10294, 2614, 8760, 15929, 10808, 26685, 17470, 8767, 19014, 12359, 26185, 1610, 17997, 8270, 27725, 13904, 589, 18000, 20052, 12379, 25695, 13408, 27235, 4195, 11366, 6761, 22135, 26232, 121, 5241, 27260, 10877, 5759, 16007, 10888, 1168, 2705, 658, 25746, 9365, 11420, 8348, 16030, 29855, 16034, 12966, 29865, 28844, 3759, 9911, 14520, 9912, 26810, 1212, 12487, 26825, 1738, 21195, 28876, 30415, 1232, 720, 26835, 25813, 16604, 20188, 25315, 17126, 9447, 15078, 18669, 17135, 20213, 21750, 4344, 4858, 27900, 25858, 4360, 2312, 14603, 14604, 12045, 23309, 6926, 20240, 1297, 5904, 23825, 5905, 29454, 13592, 6426, 17691, 14107, 3868, 26399, 19233, 5410, 10531, 17702, 1831, 5418, 14125, 17714, 10035, 10548, 3382, 13111, 1849, 11579, 10557, 27455, 11584, 3

### Identificar linhas (índices) de pacientes sobrepostos e removê-los do conjunto de treino ou do conjunto de validação


1. Crie listas dos números sobrepostos nos conjuntos de treinamento e validação.
2. Retire os registros de pacientes sobrepostos do conjunto de validação (também pode optar por retirar do conjunto de treinamento)

In [8]:
train_overlap_idxs = []
valid_overlap_idxs = []
for idx in range(n_overlap):
    train_overlap_idxs.extend(train_df.index[train_df['PatientId'] == patient_overlap[idx]].tolist())
    valid_overlap_idxs.extend(valid_df.index[valid_df['PatientId'] == patient_overlap[idx]].tolist())
    
print(f'Estes são os indices de pacientes sobrepostos no conjunto de treinamento: ')
print(f'{train_overlap_idxs}')
print(f'Estes são os indices de pacientes sobrepostos no conjunto de validação')
print(f'{valid_overlap_idxs}')

Estes são os indices de pacientes sobrepostos no conjunto de treinamento: 
[74, 112, 14, 557, 142, 187, 39, 91, 511, 598, 145, 164, 4, 176, 152, 50, 36, 193, 10, 24, 46, 604, 72, 51, 11, 95, 156, 49, 67, 635, 82, 3, 75, 124, 45, 0, 81, 63, 85, 134, 61, 163, 195, 177, 858, 7, 994, 131, 62, 171, 130, 13, 518, 150, 84, 143, 765, 21, 38, 80, 98, 148, 179, 109, 139, 87, 126, 170, 22, 1, 106, 26, 58, 190, 767, 149, 12, 157, 159, 181, 405, 100, 71, 107, 460, 894, 48, 97, 117, 146, 5, 122, 111, 128, 33, 65, 90, 44, 41, 115, 184, 6, 197, 296, 440, 789, 840, 985, 69, 123, 129, 962, 42, 160, 99, 191, 77, 101, 15, 27, 30, 110, 29, 2, 56, 102, 105, 196, 55, 57, 53, 104, 154, 138, 151, 17, 141, 878, 43, 136, 147, 64, 137, 92, 93, 178, 60, 207, 311, 674, 59, 28, 113, 68, 501, 9, 167, 18, 103, 34, 169, 32, 175, 78, 132, 168, 172, 108, 140, 189, 161, 114, 500, 125, 119, 8, 40, 54, 66, 89, 96, 31, 180, 37, 174, 16, 116, 183, 173, 94, 52, 249, 121, 182, 19, 79, 667, 165, 396, 88, 837, 118, 918, 155, 153,

In [9]:
# Excluindo ids sobrepostos
valid_df.drop(valid_overlap_idxs, inplace=True)

### Comparação de ID do paciente entre os conjuntos de treinamento e validação.

- menos registros no conjunto de validação e problema de sobreposição resolvido.

In [10]:
# Ids dos pacientes para o conjunto de validação
ids_valid = valid_df.PatientId.values
# Create a "set" datastructure of the validation set id's to identify unique id's
ids_valid_set = set(ids_valid)
print(f'Há  {len(ids_valid_set)} Ids únicos de pacientes no conjunto de validação.')

Há  2 Ids únicos de pacientes no conjunto de validação.


In [11]:
# Idnetificando conjunto sobreposto atraves de interseção de conjuntos
patient_overlap = list(ids_train_set.intersection(ids_valid_set))
n_overlap = len(patient_overlap)
print(f'Há {n_overlap} Ids únicos de pacientes no conjunto de validação.')

Há 0 Ids únicos de pacientes no conjunto de validação.
